In [1]:
import psycopg2

### Sample postgres table:

```
CREATE TABLE edge_table (
    id BIGSERIAL,
    source BIGINT,
    target BIGINT,
    cost FLOAT,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT
);
```

### Postgres connection

In [2]:
conn = psycopg2.connect(
    host="localhost",
    database="krakow_big",
    user="postgres",
    password="postgres")

### Data loading

In [3]:
conn.autocommit = True
cursor = conn.cursor() 

In [4]:
drop_table = "DROP TABLE IF EXISTS edge_table CASCADE;"

create_table = """
    CREATE TABLE edge_table (
    id BIGSERIAL,
    osmid BIGINT,
    source BIGINT,
    target BIGINT,
    length FLOAT,
    name VARCHAR,
    highway VARCHAR,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT);
"""

create_extension = """
    CREATE EXTENSION IF NOT EXISTS pgrouting CASCADE;
    CREATE EXTENSION IF NOT EXISTS postgis;
"""

create_index = """
    CREATE INDEX source_idx ON edge_table (source);
    CREATE INDEX target_idx ON edge_table (target);
"""

### Drop table (if exists)

In [5]:
cursor.execute(drop_table)

### Create extensions (pgRouting and postgis)

In [6]:
cursor.execute(create_extension)

### Create table

In [7]:
cursor.execute(create_table)

### Load data from csv

In [10]:
# TODO: remember to change path to .csv file
csv_file_name = "/home/pcend/Piotr/kod/graph-routing/data/krakow_big/edge_table_sql.csv"
copy_csv_query = "COPY edge_table(source, target, osmid, length, name, highway, x1, y1, x2, y2) FROM STDIN DELIMITER ',' CSV HEADER"

In [11]:
cursor.copy_expert(copy_csv_query, open(csv_file_name, "r"))

### Create indexes on source and target

In [12]:
cursor.execute(create_index)

### Search queries

In [13]:
ids = (356926768, 2104495834) # short (zakamycze, betel)
# ids = (13822575, 11286442329) # random small

In [22]:
dijkstra = f"""
    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

dijkstra_benchmark = f"""
    EXPLAIN ANALYZE 
    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

astar = f"""
    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

astar_benchmark = f"""
    EXPLAIN ANALYZE
    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

In [15]:
print(astar)
print(dijkstra)


    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    356926768, 2104495834,
    directed => false);


    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    356926768, 2104495834,
    directed => false);



In [16]:
cursor.execute(astar)

result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

InternalError_: invalid memory alloc request size 1080000000
CONTEXT:  SQL function "pgr_astar" statement 1


In [23]:
cursor.execute(astar_benchmark)

result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['QUERY PLAN']
('Function Scan on pgr_astar  (cost=0.25..10.25 rows=1000 width=40)',)


In [17]:
cursor.execute(dijkstra)

result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['seq', 'path_seq', 'start_vid', 'end_vid', 'node', 'edge', 'cost', 'agg_cost']
(1, 1, 356926768, 2104495834, 356926768, 12252373, 22.495, 0.0)
(2, 2, 356926768, 2104495834, 2583143012, 12540679, 30.134, 22.495)
(3, 3, 356926768, 2104495834, 2583143013, 12291041, 16.071, 52.629000000000005)
(4, 4, 356926768, 2104495834, 1364114321, 12291042, 7.768, 68.7)
(5, 5, 356926768, 2104495834, 2000518505, 12252374, 13.455, 76.468)
(6, 6, 356926768, 2104495834, 356926771, 12252375, 31.201, 89.923)
(7, 7, 356926768, 2104495834, 356926775, 12252377, 21.388, 121.124)
(8, 8, 356926768, 2104495834, 2583143014, 12540683, 15.735, 142.512)
(9, 9, 356926768, 2104495834, 2583143015, 12321370, 27.355, 158.247)
(10, 10, 356926768, 2104495834, 1752636499, 12321369, 14.562, 185.602)
(11, 11, 356926768, 2104495834, 2583137150, 12540603, 12.532, 200.16400000000002)
(12, 12, 356926768, 2104495834, 2583137151, 12540605, 13.823, 212.69600000000003)
(13, 13, 356926768, 2104495834, 2583137152, 12540607, 14.322, 226.5

In [18]:
cursor.execute(dijkstra_benchmark)

result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['QUERY PLAN']
('Function Scan on pgr_dijkstra  (cost=0.25..10.25 rows=1000 width=56) (actual time=41490.726..41490.780 rows=298 loops=1)',)
('Planning Time: 0.048 ms',)
('Execution Time: 41490.855 ms',)


### Disconnect

In [ ]:
conn.commit() 
conn.close() 